<h2>Get All the Things</h2>

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

<h2>Import the Data</h2>

In [3]:
#X is a set of parameters defined at the point of entry for each trade.
#y is a list of returns for each trade.
X = pd.read_csv("//bvt-fs-01//Users$//jwwilson//Documents//Historical//X.csv",header=None).values
y = pd.read_csv("//bvt-fs-01//Users$//jwwilson//Documents//Historical//y.csv",header=None).values.reshape(-1,1)

<h2>Prepare the Data</h2>

In [6]:
#Scale Data ()
scaler = MinMaxScaler()
scaler.fit(X)
Xs = scaler.transform(X)
Xsr = Xs[:,10:]

<h2>Define the Model<h2>

<h2>Run The Model</h2>

<h2>Analyze the Results</h2>

<h2>Functions</h2>

In [4]:
def tradeStats(results):
    
    #total profit
    total_profit = results.sum()
    number_of_trades = len(results)
    
    #probability of a win, loss or breakeven
    p_win = (results>0).sum()/len(results)
    p_loss = (results<0).sum()/len(results)
    p_be = (results==0).sum()/len(results)
    
    #Average Winner and loser
    size_win = results[results>0].sum()/ (results>0).sum()*10000
    size_loss = -results[results<0].sum()/ (results<0).sum()*10000
    rr = size_win/size_loss
    pips_without_fees = total_profit*10000
    pips_with_estimated_fees = total_profit*10000-number_of_trades*1.2
    return(total_profit,number_of_trades,p_win,p_loss,p_be,size_win,size_loss,rr,pips_without_fees,pips_with_estimated_fees)